In [1]:
import pandas as pd

In [22]:
%config Completer.use_jedi = False

In [3]:
from ast import literal_eval

In [4]:
from collections import Counter

In [5]:
from utils.basic_tokenize import tokenize

In [6]:
from gensim.utils import simple_preprocess

In [7]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [8]:
df = pd.read_csv(r"D:\data\openparliament\df_climate.csv")
df.drop('Unnamed: 0', axis=1, inplace=True)

In [9]:
df['text_en']= df.text_en.apply(lambda x: x.replace('Mr. Speaker,',' '))

In [10]:
df['tokens'] = df.tokenized_content.apply(literal_eval)

In [11]:
%%time
df['tokens'] = df.text_en.apply(lambda x: simple_preprocess(x, deacc=True))

Wall time: 5.29 s


In [12]:
def bigrams(sentence):
    part1 = [i+ "_"+j for i,j in zip(sentence[::2], sentence[1::2])]
    part2 = [i+ "_"+j for i,j in zip(sentence[1::2], sentence[2::2])]
    
    part1.extend(part2)
    return part1

In [13]:
a = ['a','b','c']
b = [1,2,3]

In [14]:
list(zip(a,b))

[('a', 1), ('b', 2), ('c', 3)]

In [15]:
df['bigrams'] = df.tokens.apply(bigrams)

In [16]:
df.bigrams.head()

0    [last_week, more_than, six_million, canadians_...
1    [last_week, more_than, six_million, canadians_...
2    [last_week, more_than, six_million, canadians_...
3    [it_is, privilege_to, be_able, to_speak, to_th...
4    [it_is, privilege_to, be_able, to_speak, to_th...
Name: bigrams, dtype: object

In [17]:
bigram_counts = dict()
template_counter = Counter()
pids = list(set(df.politician_id))
for pol in pids:
    bigram_counts[pol] = template_counter.copy()

In [18]:
def update_counts(row):
    bigrams = row.loc['bigrams']
    if bigram_counts.get(row.loc['politician_id'])==None:
        bigram_counts[row.loc['politician_id']] = template_counter.copy()
        bigram_counts[row.loc['politician_id']].update(bigrams)
    else:
        bigram_counts[row.loc['politician_id']].update(bigrams)

In [19]:
df.apply(update_counts,axis=1)

0        None
1        None
2        None
3        None
4        None
         ... 
19655    None
19656    None
19657    None
19658    None
19659    None
Length: 19660, dtype: object

In [20]:
for key, counter in bigram_counts.items():
    print('politician #:', key)
    print(counter.most_common(4))

politician #: 10240
[('we_are', 3), ('of_the', 2), ('this_process', 2), ('following_the', 1)]
politician #: 1
[('of_the', 8), ('and_air', 8), ('new_government', 8), ('canada_new', 8)]
politician #: 2
[('will_be', 8), ('the_member', 4), ('happy_to', 4), ('know_that', 4)]
politician #: 5
[('carbon_tax', 22), ('the_liberal', 18), ('prime_minister', 12), ('climate_change', 12)]
politician #: 6
[('of_the', 184), ('we_are', 176), ('to_the', 148), ('that_we', 140)]
politician #: 7
[('of_the', 52), ('carbon_tax', 52), ('federal_government', 36), ('the_federal', 36)]
politician #: 9
[('we_are', 106), ('we_have', 94), ('of_the', 86), ('the_ndp', 72)]
politician #: 10
[('blue_green', 21), ('green_algae', 21), ('this_bill', 15), ('the_government', 12)]
politician #: 11
[('of_the', 29), ('it_is', 25), ('we_are', 24), ('the_government', 19)]
politician #: 12
[('in_the', 12), ('of_the', 12), ('st_lawrence', 8), ('climate_change', 8)]
politician #: 10253
[('energy_efficiency', 3), ('our_climate', 2), 

load politician info

In [21]:
pol_info = pd.read_csv(r"D:\data\openparliament\politician_info.csv",index_col='Unnamed: 0')

note: how to sum arbitrarily many counters

In [23]:
pol_group = pol_info.groupby('party_id')

In [30]:
conservatives = df[df.party_id==1].politician_id.unique()

In [36]:
%%time
conservative_bigrams = sum([bigram_counts[p] for p in conservatives],Counter())

Wall time: 11.2 s


In [38]:
conservative_bigrams.most_common(15)

[('of_the', 7169),
 ('in_the', 4516),
 ('it_is', 3653),
 ('we_are', 3539),
 ('the_government', 3102),
 ('that_is', 3031),
 ('to_the', 3017),
 ('the_environment', 3014),
 ('we_have', 2811),
 ('carbon_tax', 2679),
 ('that_the', 2600),
 ('on_the', 2409),
 ('and_the', 2218),
 ('for_the', 2157),
 ('the_liberals', 2087)]